# Insurance cost prediction using linear regression

In this assignment we're going to use information like a person's age, sex, BMI, no. of children and smoking habit to predict the price of yearly medical bills. This kind of model is useful for insurance companies to determine the yearly insurance premium for a person. The dataset for this problem is taken from: https://www.kaggle.com/mirichoi0218/insurance


We will create a model with the following steps:
1. Download and explore the dataset
2. Prepare the dataset for training
3. Create a linear regression model
4. Train the model to fit the data
5. Make predictions using the trained model


This assignment builds upon the concepts from the first 2 lectures. It will help to review these Jupyter notebooks:
- PyTorch basics: https://jovian.ml/aakashns/01-pytorch-basics
- Linear Regression: https://jovian.ml/aakashns/02-linear-regression
- Logistic Regression: https://jovian.ml/aakashns/03-logistic-regression
- Linear regression (minimal): https://jovian.ml/aakashns/housing-linear-minimal
- Logistic regression (minimal): https://jovian.ml/aakashns/mnist-logistic-minimal

As you go through this notebook, you will find a **???** in certain places. Your job is to replace the **???** with appropriate code or values, to ensure that the notebook runs properly end-to-end . In some cases, you'll be required to choose some hyperparameters (learning rate, batch size etc.). Try to experiment with the hypeparameters to get the lowest loss.


In [83]:
# Uncomment and run the commands below if imports fail
# !conda install numpy pytorch torchvision cpuonly -c pytorch -y
# !pip install matplotlib --upgrade --quiet
!pip install jovian --upgrade --quiet

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [84]:
import torch
import jovian
import torchvision
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torch.utils.data import DataLoader, TensorDataset, random_split

In [85]:
project_name='02-insurance-linear-regression' # will be used by jovian.commit

## Step 1: Download and explore the data

Let us begin by downloading the data. We'll use the `download_url` function from PyTorch to get the data as a CSV (comma-separated values) file. 

In [86]:
DATASET_URL = "https://hub.jovian.ml/wp-content/uploads/2020/05/insurance.csv"
DATA_FILENAME = "insurance.csv"
download_url(DATASET_URL, '.')

Using downloaded and verified file: ./insurance.csv


To load the dataset into memory, we'll use the `read_csv` function from the `pandas` library. The data will be loaded as a Pandas dataframe. See this short tutorial to learn more: https://data36.com/pandas-tutorial-1-basics-reading-data-files-dataframes-data-selection/

In [87]:
dataframe_raw = pd.read_csv(DATA_FILENAME)
dataframe_raw.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


We're going to do a slight customization of the data, so that you every participant receives a slightly different version of the dataset. Fill in your name below as a string (enter at least 5 characters)

In [88]:
your_name = 'Mridul'

The `customize_dataset` function will customize the dataset slightly using your name as a source of random numbers.

In [89]:
def customize_dataset(dataframe_raw, rand_str):
    dataframe = dataframe_raw.copy(deep=True)
    # drop some rows
    dataframe = dataframe.sample(int(0.95*len(dataframe)), random_state=int(ord(rand_str[0])))
    # scale input
    dataframe.bmi = dataframe.bmi * ord(rand_str[1])/100.
    # scale target
    dataframe.charges = dataframe.charges * ord(rand_str[2])/100.
    # drop column
    if ord(rand_str[3]) % 2 == 1:
        dataframe = dataframe.drop(['region'], axis=1)
    return dataframe

In [90]:
dataframe = customize_dataset(dataframe_raw, your_name)
dataframe.head()

,age,sex,bmi,children,smoker,region,charges
27,55,female,37.3635,2,no,northwest,12882.063862
38,35,male,41.8038,1,yes,northeast,41762.990115
441,33,female,38.1900,0,yes,southwest,38933.340600
1068,63,male,24.6924,1,no,northwest,15067.347120
1310,42,male,29.9991,1,no,northwest,7287.955343


Let us answer some basic questions about the dataset. 


**Q: How many rows does the dataset have?**

In [91]:
num_rows = len(dataframe)
print(num_rows)

1271


**Q: How many columns doe the dataset have**

In [92]:
num_cols = len(dataframe.columns)
print(num_cols)

7


**Q: What are the column titles of the input variables?**

In [93]:
input_cols = ['age','sex','bmi','children','smoker']

**Q: Which of the input columns are non-numeric or categorial variables ?**

Hint: `sex` is one of them. List the columns that are not numbers.

In [94]:
categorical_cols = ['sex','smoker']

**Q: What are the column titles of output/target variable(s)?**

In [95]:
output_cols = ['charges']

**Q: (Optional) What is the minimum, maximum and average value of the `charges` column? Can you show the distribution of values in a graph?**
Use this data visualization cheatsheet for referece: https://jovian.ml/aakashns/dataviz-cheatsheet

In [96]:
min = dataframe['charges'].min()
max = dataframe['charges'].max()
avg = dataframe['charges'].mean()

min, max, avg

(1177.967595, 66958.9494105, 13926.807592908699)

Remember to commit your notebook to Jovian after every step, so that you don't lose your work.

In [ ]:
jovian.commit(project=project_name, environment=None)

## Step 2: Prepare the dataset for training

We need to convert the data from the Pandas dataframe into a PyTorch tensors for training. To do this, the first step is to convert it numpy arrays. If you've filled out `input_cols`, `categorial_cols` and `output_cols` correctly, this following function will perform the conversion to numpy arrays.

In [97]:
def dataframe_to_arrays(dataframe):
    # Make a copy of the original dataframe
    dataframe1 = dataframe.copy(deep=True)
    # Convert non-numeric categorical columns to numbers
    for col in categorical_cols:
        dataframe1[col] = dataframe1[col].astype('category').cat.codes
    # Extract input & outupts as numpy arrays
    inputs_array = dataframe1[input_cols].to_numpy()
    targets_array = dataframe1[output_cols].to_numpy()
    return inputs_array, targets_array

Read through the [Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html) to understand how we're converting categorical variables into numbers.

In [98]:
inputs_array, targets_array = dataframe_to_arrays(dataframe)
inputs_array, targets_array

(array([[55.    ,  0.    , 37.3635,  2.    ,  0.    ],
        [35.    ,  1.    , 41.8038,  1.    ,  1.    ],
        [33.    ,  0.    , 38.19  ,  0.    ,  1.    ],
        ...,
        [29.    ,  1.    , 42.5106,  2.    ,  0.    ],
        [26.    ,  1.    , 27.018 ,  2.    ,  0.    ],
        [18.    ,  0.    , 36.3888,  0.    ,  0.    ]]),
 array([[12882.0638625],
        [41762.990115 ],
        [38933.3406   ],
        ...,
        [ 4261.021905 ],
        [ 3658.54755  ],
        [ 2316.27984  ]]))

**Q: Convert the numpy arrays `inputs_array` and `targets_array` into PyTorch tensors. Make sure that the data type is `torch.float32`.**

In [99]:
inputs = torch.from_numpy(inputs_array)
inputs = inputs.type(torch.float32)
targets = torch.from_numpy(targets_array)
targets = targets.type(torch.float32)

In [100]:
inputs.dtype, targets.dtype

(torch.float32, torch.float32)

Next, we need to create PyTorch datasets & data loaders for training & validation. We'll start by creating a `TensorDataset`.

In [101]:
dataset = TensorDataset(inputs, targets)

**Q: Pick a number between `0.1` and `0.2` to determine the fraction of data that will be used for creating the validation set. Then use `random_split` to create training & validation datasets. **

In [102]:
val_percent = 0.17
val_size = int(num_rows * val_percent)
train_size = num_rows - val_size


train_ds, val_ds = random_split(dataset, [train_size, val_size])

In [103]:
len(val_ds)

216

Finally, we can create data loaders for training & validation.

**Q: Pick a batch size for the data loader.**

In [104]:
batch_size = 30

In [106]:
train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size)

Let's look at a batch of data to verify everything is working fine so far.

In [107]:
for xb, yb in train_loader:
    print("inputs:", xb)
    print("targets:", yb)
    break

inputs: tensor([[25.0000,  1.0000, 51.9156,  2.0000,  1.0000],
        [29.0000,  0.0000, 35.5224,  0.0000,  0.0000],
        [47.0000,  0.0000, 26.9040,  1.0000,  0.0000],
        [18.0000,  0.0000, 45.8964,  0.0000,  0.0000],
        [58.0000,  0.0000, 37.5801,  0.0000,  0.0000],
        [32.0000,  1.0000, 32.9232,  0.0000,  0.0000],
        [39.0000,  1.0000, 48.6267,  0.0000,  0.0000],
        [42.0000,  1.0000, 28.3404,  0.0000,  0.0000],
        [25.0000,  1.0000, 29.3436,  0.0000,  0.0000],
        [38.0000,  1.0000, 33.3564,  2.0000,  0.0000],
        [54.0000,  1.0000, 28.6140,  3.0000,  1.0000],
        [58.0000,  1.0000, 41.1312,  0.0000,  0.0000],
        [37.0000,  1.0000, 33.9720,  0.0000,  0.0000],
        [47.0000,  0.0000, 38.0133,  0.0000,  0.0000],
        [60.0000,  0.0000, 40.0140,  0.0000,  0.0000],
        [19.0000,  0.0000, 41.6955,  0.0000,  0.0000],
        [31.0000,  1.0000, 23.2560,  0.0000,  0.0000],
        [26.0000,  1.0000, 23.7120,  0.0000,  0.0000],
  

Let's save our work by committing to Jovian.

In [ ]:
jovian.commit(project=project_name, environment=None)

## Step 3: Create a Linear Regression Model

Our model itself is a fairly straightforward linear regression (we'll build more complex models in the next assignment). 


In [108]:
input_size = len(input_cols)
output_size = len(output_cols)

In [109]:
input_size, output_size

(5, 1)

**Q: Complete the class definition below by filling out the constructor (`__init__`), `forward`, `training_step` and `validation_step` methods.**

Hint: Think carefully about picking a good loss fuction (it's not cross entropy). Maybe try 2-3 of them and see which one works best. See https://pytorch.org/docs/stable/nn.functional.html#loss-functions

In [110]:
class InsuranceModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(input_size, output_size)        # fill this (hint: use input_size & output_size defined above)
        
    def forward(self, xb):
        xb = xb.reshape(-1, 5)
        out =  self.linear(xb)        # fill this
        return out
    
    def training_step(self, batch):
        inputs, targets = batch 
        # Generate predictions
        out = self(inputs)          
        # Calcuate loss
        loss = F.l1_loss(out, targets)                        # fill this
        return loss
    
    def validation_step(self, batch):
        inputs, targets = batch
        # Generate predictions
        out = self(inputs)
        # Calculate loss
        loss = F.l1_loss(out, targets)                          # fill this    
        return {'val_loss': loss.detach()}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        return {'val_loss': epoch_loss.item()}
    
    def epoch_end(self, epoch, result, num_epochs):
        # Print result every 20th epoch
        if (epoch+1) % 20 == 0 or epoch == num_epochs-1:
            print("Epoch [{}], val_loss: {:.4f}".format(epoch+1, result['val_loss']))

Let us create a model using the `InsuranceModel` class. You may need to come back later and re-run the next cell to reinitialize the model, in case the loss becomes `nan` or `infinity`.

In [111]:
model = InsuranceModel()

Let's check out the weights and biases of the model using `model.parameters`.

In [112]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.1297,  0.3630,  0.1029, -0.3296,  0.0602]], requires_grad=True),
 Parameter containing:
 tensor([0.0286], requires_grad=True)]

One final commit before we train the model.

In [ ]:
jovian.commit(project=project_name, environment=None)

## Step 4: Train the model to fit the data

To train our model, we'll use the same `fit` function explained in the lecture. That's the benefit of defining a generic training loop - you can use it for any problem.

In [113]:
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result, epochs)
        history.append(result)
    return history

**Q: Use the `evaluate` function to calculate the loss on the validation set before training.**

In [114]:
result = evaluate(model,val_loader)
print(result)

{'val_loss': 14016.18359375}



We are now ready to train the model. You may need to run the training loop many times, for different number of epochs and with different learning rates, to get a good result. Also, if your loss becomes too large (or `nan`), you may have to re-initialize the model by running the cell `model = InsuranceModel()`. Experiment with this for a while, and try to get to as low a loss as possible.

**Q: Train the model 4-5 times with different learning rates & for different number of epochs.**

Hint: Vary learning rates by orders of 10 (e.g. `1e-2`, `1e-3`, `1e-4`, `1e-5`, `1e-6`) to figure out what works.

In [115]:
epochs = 1000
lr = 1e-4
history1 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 13813.5615
Epoch [40], val_loss: 13611.0615
Epoch [60], val_loss: 13408.4297
Epoch [80], val_loss: 13205.8047
Epoch [100], val_loss: 13003.2451
Epoch [120], val_loss: 12800.5391
Epoch [140], val_loss: 12597.8008
Epoch [160], val_loss: 12395.9805
Epoch [180], val_loss: 12195.9609
Epoch [200], val_loss: 11997.7764
Epoch [220], val_loss: 11806.0791
Epoch [240], val_loss: 11619.6006
Epoch [260], val_loss: 11437.4756
Epoch [280], val_loss: 11260.7822
Epoch [300], val_loss: 11092.2148
Epoch [320], val_loss: 10929.9141
Epoch [340], val_loss: 10775.0605
Epoch [360], val_loss: 10624.6699
Epoch [380], val_loss: 10479.0674
Epoch [400], val_loss: 10339.4766
Epoch [420], val_loss: 10209.9990
Epoch [440], val_loss: 10082.9307
Epoch [460], val_loss: 9957.4883
Epoch [480], val_loss: 9834.0723
Epoch [500], val_loss: 9714.0195
Epoch [520], val_loss: 9599.2383
Epoch [540], val_loss: 9487.1230
Epoch [560], val_loss: 9382.6455
Epoch [580], val_loss: 9283.9492
Epoch [600], val_loss: 91

In [116]:
epochs = 500
lr = 1e-1
history2 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 6907.1460
Epoch [40], val_loss: 6866.0215
Epoch [60], val_loss: 6855.4531
Epoch [80], val_loss: 6856.9277
Epoch [100], val_loss: 6849.7681
Epoch [120], val_loss: 6835.0361
Epoch [140], val_loss: 6828.9893
Epoch [160], val_loss: 6824.6104
Epoch [180], val_loss: 6818.6968
Epoch [200], val_loss: 6808.1455
Epoch [220], val_loss: 6819.8892
Epoch [240], val_loss: 6799.4370
Epoch [260], val_loss: 6798.9990
Epoch [280], val_loss: 6796.8232
Epoch [300], val_loss: 6785.2944
Epoch [320], val_loss: 6780.7876
Epoch [340], val_loss: 6774.0366
Epoch [360], val_loss: 6769.6699
Epoch [380], val_loss: 6766.9551
Epoch [400], val_loss: 6767.5176
Epoch [420], val_loss: 6758.9160
Epoch [440], val_loss: 6760.4604
Epoch [460], val_loss: 6753.7793
Epoch [480], val_loss: 6742.5952
Epoch [500], val_loss: 6740.6177


In [117]:
epochs = 1000
lr = 1e-1
history3 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 6736.6172
Epoch [40], val_loss: 6770.6436
Epoch [60], val_loss: 6731.9395
Epoch [80], val_loss: 6723.5464
Epoch [100], val_loss: 6724.3071
Epoch [120], val_loss: 6718.9468
Epoch [140], val_loss: 6719.2617
Epoch [160], val_loss: 6718.4502
Epoch [180], val_loss: 6709.0391
Epoch [200], val_loss: 6708.7197
Epoch [220], val_loss: 6721.1401
Epoch [240], val_loss: 6704.4961
Epoch [260], val_loss: 6710.4795
Epoch [280], val_loss: 6691.1494
Epoch [300], val_loss: 6686.8267
Epoch [320], val_loss: 6679.8203
Epoch [340], val_loss: 6678.3472
Epoch [360], val_loss: 6676.9526
Epoch [380], val_loss: 6671.2793
Epoch [400], val_loss: 6665.4907
Epoch [420], val_loss: 6667.4429
Epoch [440], val_loss: 6663.0928
Epoch [460], val_loss: 6667.9619
Epoch [480], val_loss: 6653.8506
Epoch [500], val_loss: 6656.4136
Epoch [520], val_loss: 6644.5049
Epoch [540], val_loss: 6646.1504
Epoch [560], val_loss: 6641.0396
Epoch [580], val_loss: 6634.9502
Epoch [600], val_loss: 6632.9609
Epoch [620], v

In [118]:
epochs = 1000
lr = 1
history4 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 6836.4946
Epoch [40], val_loss: 7264.4990
Epoch [60], val_loss: 6577.8501
Epoch [80], val_loss: 7064.8867
Epoch [100], val_loss: 8148.1274
Epoch [120], val_loss: 6896.7041
Epoch [140], val_loss: 6375.9077
Epoch [160], val_loss: 6453.9624
Epoch [180], val_loss: 6529.4800
Epoch [200], val_loss: 6389.9023
Epoch [220], val_loss: 6238.0522
Epoch [240], val_loss: 6522.8696
Epoch [260], val_loss: 6642.6396
Epoch [280], val_loss: 8100.3921
Epoch [300], val_loss: 6605.1567
Epoch [320], val_loss: 6257.8232
Epoch [340], val_loss: 6265.6948
Epoch [360], val_loss: 6273.8096
Epoch [380], val_loss: 6537.9756
Epoch [400], val_loss: 6001.3550
Epoch [420], val_loss: 6260.5020
Epoch [440], val_loss: 5938.3838
Epoch [460], val_loss: 5966.3076
Epoch [480], val_loss: 7615.2183
Epoch [500], val_loss: 6818.4741
Epoch [520], val_loss: 6451.1211
Epoch [540], val_loss: 6337.3569
Epoch [560], val_loss: 6496.4746
Epoch [580], val_loss: 5840.9844
Epoch [600], val_loss: 5783.3828
Epoch [620], v

In [124]:
epochs = 1000
lr = 1e-1
history5 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 4407.4819
Epoch [40], val_loss: 4406.7544
Epoch [60], val_loss: 4402.8213
Epoch [80], val_loss: 4422.4761
Epoch [100], val_loss: 4401.5239
Epoch [120], val_loss: 4398.5435
Epoch [140], val_loss: 4392.9556
Epoch [160], val_loss: 4403.0933
Epoch [180], val_loss: 4393.5024
Epoch [200], val_loss: 4387.9316
Epoch [220], val_loss: 4383.8633
Epoch [240], val_loss: 4389.8477
Epoch [260], val_loss: 4381.8608
Epoch [280], val_loss: 4378.6221
Epoch [300], val_loss: 4381.0439
Epoch [320], val_loss: 4377.6250
Epoch [340], val_loss: 4370.9795
Epoch [360], val_loss: 4384.9019
Epoch [380], val_loss: 4376.8867
Epoch [400], val_loss: 4420.6226
Epoch [420], val_loss: 4361.6812
Epoch [440], val_loss: 4395.4102
Epoch [460], val_loss: 4357.4966
Epoch [480], val_loss: 4370.8301
Epoch [500], val_loss: 4384.5840
Epoch [520], val_loss: 4353.5381
Epoch [540], val_loss: 4365.0806
Epoch [560], val_loss: 4349.8320
Epoch [580], val_loss: 4347.2285
Epoch [600], val_loss: 4346.5781
Epoch [620], v

**Q: What is the final validation loss of your model?**

In [125]:
val_loss = 4309.0977

Let's log the final validation loss to Jovian and commit the notebook

In [126]:
jovian.log_metrics(val_loss=val_loss)

[jovian] Metrics logged.


In [127]:
jovian.commit(project=project_name, environment=None)

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
[jovian] Updating notebook "mridul-chavan/02-insurance-linear-regression" on https://jovian.ml/
[jovian] Uploading notebook..
[jovian] Attaching records (metrics, hyperparameters, dataset etc.)
[jovian] Committed successfully! https://jovian.ml/mridul-chavan/02-insurance-linear-regression


'https://jovian.ml/mridul-chavan/02-insurance-linear-regression'

Now scroll back up, re-initialize the model, and try different set of values for batch size, number of epochs, learning rate etc. Commit each experiment and use the "Compare" and "View Diff" options on Jovian to compare the different results.

## Step 5: Make predictions using the trained model

**Q: Complete the following function definition to make predictions on a single input**

In [128]:
def predict_single(input, target, model):
    inputs = input.unsqueeze(0)
    predictions = model(inputs)              # fill this
    prediction = predictions[0].detach()
    print("Input:", input)
    print("Target:", target)
    print("Prediction:", prediction)

In [129]:
input, target = val_ds[0]
predict_single(input, target, model)

Input: tensor([48.0000,  0.0000, 31.0821,  1.0000,  0.0000])
Target: tensor([9919.6133])
Prediction: tensor([10083.9980])


In [130]:
input, target = val_ds[10]
predict_single(input, target, model)

Input: tensor([52.0000,  0.0000, 20.9019,  0.0000,  0.0000])
Target: tensor([10490.5898])
Prediction: tensor([10957.0645])


In [131]:
input, target = val_ds[23]
predict_single(input, target, model)

Input: tensor([52.0000,  1.0000, 54.4236,  1.0000,  0.0000])
Target: tensor([10236.3564])
Prediction: tensor([10271.7324])


Are you happy with your model's predictions? Try to improve them further.

## (Optional) Step 6: Try another dataset & blog about it

While this last step is optional for the submission of your assignment, we highly recommend that you do it. Try to clean up & replicate this notebook (or [this one](https://jovian.ml/aakashns/housing-linear-minimal), or [this one](https://jovian.ml/aakashns/mnist-logistic-minimal) ) for a different linear regression or logistic regression problem. This will help solidify your understanding, and give you a chance to differentiate the generic patters in machine learning from problem-specific details.

Here are some sources to find good datasets:

- https://lionbridge.ai/datasets/10-open-datasets-for-linear-regression/
- https://www.kaggle.com/rtatman/datasets-for-regression-analysis
- https://archive.ics.uci.edu/ml/datasets.php?format=&task=reg&att=&area=&numAtt=&numIns=&type=&sort=nameUp&view=table
- https://people.sc.fsu.edu/~jburkardt/datasets/regression/regression.html
- https://archive.ics.uci.edu/ml/datasets/wine+quality
- https://pytorch.org/docs/stable/torchvision/datasets.html

We also recommend that you write a blog about your approach to the problem. Here is a suggested structure for your post (feel free to experiment with it):

- Interesting title & subtitle
- Overview of what the blog covers (which dataset, linear regression or logistic regression, intro to PyTorch)
- Downloading & exploring the data
- Preparing the data for training
- Creating a model using PyTorch
- Training the model to fit the data
- Your thoughts on how to experiment with different hyperparmeters to reduce loss
- Making predictions using the model

As with the previous assignment, you can [embed Juptyer notebook cells & outputs from Jovian](https://medium.com/jovianml/share-and-embed-jupyter-notebooks-online-with-jovian-ml-df709a03064e) into your blog. 

Don't forget to share your work on the forum: https://jovian.ml/forum/t/share-your-work-here-assignment-2/4931

In [ ]:
jovian.commit(project=project_name, environment=None)
jovian.commit(project=project_name, environment=None) # try again, kaggle fails sometimes

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
[jovian] Updating notebook "mridul-chavan/02-insurance-linear-regression" on https://jovian.ml/
[jovian] Uploading notebook..
[jovian] Attaching records (metrics, hyperparameters, dataset etc.)
[jovian] Committed successfully! https://jovian.ml/mridul-chavan/02-insurance-linear-regression


<IPython.core.display.Javascript object>